<a href="https://colab.research.google.com/github/tgusmao/programacao_ds/blob/master/TrabalhoPratico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Prático de Programação para Ciências de Dados**

# Importação da Base de Dados

In [ ]:
import pandas as pd

Iniciando abaixo a leitura do arquivo CSV local (localizado no Google Drive)



Dupla1: Tammy e Thalita

pd.read_csv( string, sep = String, skiprows = int, encoding = string especifica)

1.   NOme do arquivo.
2.   String que separa os dados no arquivo.
3. Quantidade de linhas a serem puladas (linhas iniciais, como cabeçalho). 
4. Especificação de como o arquivo está codificado (utf-8 ou latin-1). 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_duvidas = pd.read_csv('/content/drive/My Drive/dados-curso-completo.csv', sep = ',', skiprows = 0, encoding = 'utf-8')

In [ ]:
df_duvidas

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
0,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
1,2020-03-18 03:18:14,Seu Zé da Farmácia,seuzedafarmacia,"Em ação preventiva, o Governo do Estado suspen...",0,56,pt,NaN,NaN,NaN,NaN,NaN,[]
2,2020-03-29 12:09:27,Marília Oliveira 🇧🇷,Marioliveira_gb,Incrível como os defensores do fim do distanci...,0,5,pt,NaN,NaN,NaN,NaN,NaN,[]
3,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.8778,-15.77691,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658820,2020-03-13 17:26:18,broca,whoisdrill,oms: tamo tendo pandemia evitar\n conglomerado...,0,361,pt,NaN,NaN,NaN,NaN,NaN,[]
1658821,2020-05-12 13:02:34,Renato 🇧🇷,VerdadeExpress,"@OsmarTerra Sr. Osmar Terra, gostaria de ouvir...",0,4066,pt,NaN,NaN,NaN,NaN,NaN,[]
1658822,2020-03-19 18:16:30,eugeniocosta8,EugenioCosta8,CIENTISTA DE HARVARD PREVÊ QUE ATÉ 70% DO MUND...,3,5307,pt,NaN,NaN,NaN,NaN,NaN,['CORONAVIRUS']
1658823,2020-04-07 18:14:45,Wagner Mota - Seriedade + Geek,wagnermota75,"#BolsonaroTemRazaoSim porque, quando a prefeit...",0,1000,pt,NaN,NaN,NaN,NaN,NaN,['BolsonaroTemRazaoSim']


In [ ]:
df_duvidas.isnull().sum()

data                0
usuario            57
apelido             0
texto               0
retweet             0
seguidores          0
idioma              0
lugar         1584220
pais          1584225
sigla         1584230
latitude      1650105
longitude     1650105
hashtags            0
dtype: int64

# Tratamento de Dados

Renomeando Colunas:

dataframe.rename(columns = **dict**)


1.   dict = {'nome_coluna_antiga' : 'nome_coluna_nova'}
2.   Item da lista



In [ ]:
df_duvidas = df_duvidas.rename(columns = {'Data':'data','Usuario':'usuario', 'Apelido':'apelido', 'Texto':'texto', 'Retweet':'retweet', 'Seguidores':'seguidores', 'Idioma':'idioma', 'Lugar':'lugar', 'Pais':'pais', 'Sigla':'sigla_pais', 'Latitude':'lat', 'Longitude':'long', 'Hashtags':'hashtags'})

In [ ]:
df_duvidas.shape

(1658825, 13)

In [ ]:
df_duvidas = df_duvidas.dropna(subset=['latitude'])

In [ ]:
df_duvidas.shape

(8720, 13)

In [ ]:
df_duvidas.isnull().sum()

data           0
usuario        0
apelido        0
texto          0
retweet        0
seguidores     0
idioma         0
lugar         11
pais          12
sigla         13
latitude       0
longitude      0
hashtags       0
dtype: int64

In [ ]:
df_duvidas

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.877800,-15.776910,[]
261,2020-04-16 00:34:54,Felipe Silva,felipesilvasr,A secretaria de Saúde da Paraíba confirmou nes...,0,150,pt,Taperoá,Brazil,BR,-36.825000,-7.162778,[]
265,2020-05-03 13:48:52,Bruno Fortes,Brunaogf,Domingão taí! ☀️\nEm casa! 😐😬\nVai passar! 🙏🏼\...,0,120,pt,Fortaleza,Brazil,BR,-38.526600,-3.729300,"['emcasa', 'isolamentosocial', 'saude', 'quero..."
289,2020-03-31 17:52:37,jairo santos souza,soi_jairo,"Seguindo orientação do Presidente Bolsonaro, o...",0,22,pt,Euclides da Cunha,Brazil,BR,-39.014620,-10.505250,[]
477,2020-03-24 17:27:29,Carolina Porne,carolinaporne,#5 - Isolamento social não é desculpa para fic...,0,373,pt,Guarulhos,Brazil,BR,-46.495600,-23.445600,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658284,2020-04-16 19:45:29,Bruno Agrofoglio Ferreira,agrofoglio,Criei um bot para o Telegram raspando os dados...,0,49,pt,Paulínia,Brazil,BR,-47.203010,-22.757163,[]
1658503,2020-05-31 12:38:32,BARÃO DE CEARA MIRIM,baraocearamirim,Serra da Formiga em Riachuelo RN\nDestino marc...,0,808,pt,Riachuelo,Brazil,BR,-35.823770,-5.815792,[]
1658514,2020-03-23 23:48:19,Diego Pelizari,diegopelizari,#Repost de respeito da parceira agroschoolbras...,0,174,pt,São Félix do Araguaia,Brazil,BR,-53.000000,-11.000000,['Repost']
1658760,2020-05-12 09:30:34,Hamilton Valerio,hvalerio,Mais um amanhecer em isolamento social. em Pra...,0,176,pt,Jaboatão dos Guararapes,Brazil,BR,-34.919555,-8.198139,[]
